# FCPM Workflow 1: Simulation + Reconstruction

This notebook demonstrates the complete FCPM workflow when you have a **known director field**:

1. Load or create a director field (ground truth)
2. Simulate FCPM measurements
3. Add realistic noise
4. Reconstruct the director field
5. Evaluate reconstruction quality

**Use case**: Testing reconstruction algorithms, validating methods, understanding noise sensitivity.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import fcpm

print(f"FCPM Library version: {fcpm.__version__}")

FCPM Library version: 1.1.0


## Option A: Quick Workflow (One Function Call)

The simplest way to run the full pipeline is using `run_simulation_reconstruction()`:

In [6]:
# Configure the workflow
config = fcpm.WorkflowConfig(
    crop_size=(64, 64, 32),    # Crop to manageable size
    noise_level=0.03,           # Add 3% noise
    noise_model='mixed',        # Gaussian + Poisson (realistic)
    noise_seed=42,              # Reproducible results
    fix_signs=True,             # Apply sign optimization
    save_plots=True,            # Generate all plots
    save_data=True,             # Save NPZ files
    verbose=True,
)

# Run the full workflow
# Replace with your director file path
results = fcpm.run_simulation_reconstruction(
    director_source='../Archive/Clean Simulation/Data/S1HighResol_6-10s.npz',
    output_dir='../output/notebook_workflow1',
    config=config
)

FCPM Simulation + Reconstruction Workflow

[1/6] Loading director field...
      Shape: (500, 500, 50)

[2/6] Cropping to (64, 64, 32)...
      New shape: (64, 64, 32)

[3/6] Simulating FCPM measurements...
      Generated 4 polarization angles

[4/6] Adding mixed noise (level=0.03)...

[5/6] Reconstructing from FCPM...
Combined Sign Optimization

Phase 1: Chain propagation...
  After chain propagation: energy = 60016.12

Phase 2: Local flip refinement...
Initial gradient energy: 60016.12
  Iter 0: flipped 25 voxels, energy = 60016.12
  Iter 1: flipped 10 voxels, energy = 59799.87
  Iter 2: flipped 5 voxels, energy = 59717.32
  Iter 3: flipped 6 voxels, energy = 59714.86
  Iter 4: flipped 7 voxels, energy = 59709.56
  Iter 10: flipped 10 voxels, energy = 59733.44
  Iter 20: flipped 10 voxels, energy = 59733.44
  Iter 30: flipped 10 voxels, energy = 59733.44
  Iter 40: flipped 10 voxels, energy = 59733.44

[6/6] Evaluating reconstruction...

Results Summary
  Angular error (mean):   17.

In [7]:
# Access results
print("\n=== Results Summary ===")
print(f"Reconstructed shape: {results.director_recon.shape}")
print(f"Angular error (mean): {results.metrics['angular_error_mean_deg']:.2f} degrees")
print(f"Angular error (median): {results.metrics['angular_error_median_deg']:.2f} degrees")
print(f"Intensity RMSE: {results.metrics['intensity_rmse_mean']:.4f}")


=== Results Summary ===
Reconstructed shape: (64, 64, 32)
Angular error (mean): 17.57 degrees
Angular error (median): 18.66 degrees
Intensity RMSE: 0.0345


## Option B: Step-by-Step Workflow

For more control, you can run each step individually:

### Step 1: Create or Load a Director Field

In [8]:
# Option 1: Create a synthetic cholesteric liquid crystal
director_gt = fcpm.create_cholesteric_director(
    shape=(64, 64, 32),
    pitch=8.0,    # Helix pitch in voxels
    axis='z'      # Helix axis
)

# Option 2: Load from file (uncomment to use)
# director_gt = fcpm.load_director('your_director_file.npz')
# director_gt = fcpm.crop_director_center(director_gt, size=(64, 64, 32))

print(f"Director field shape: {director_gt.shape}")
print(f"Director components: nx, ny, nz")

Director field shape: (64, 64, 32)
Director components: nx, ny, nz


In [9]:
# Visualize the ground truth director
z_mid = director_gt.shape[2] // 2

fig, ax = plt.subplots(figsize=(8, 8))
fcpm.plot_director_slice(director_gt, z_idx=z_mid, step=2, ax=ax, title='Ground Truth Director')
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42033/3246140615.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Step 2: Simulate FCPM Measurements

In [10]:
# Simulate FCPM intensities at 4 polarization angles
I_fcpm_clean = fcpm.simulate_fcpm(director_gt)

print(f"Simulated {len(I_fcpm_clean)} polarization angles:")
for angle in I_fcpm_clean.keys():
    print(f"  {np.degrees(angle):.1f} degrees")

Simulated 4 polarization angles:
  0.0 degrees
  45.0 degrees
  90.0 degrees
  135.0 degrees


In [11]:
# Visualize clean FCPM intensities
fig = fcpm.plot_fcpm_intensities(I_fcpm_clean, z_idx=z_mid)
fig.suptitle('Clean FCPM Intensities', y=1.02)
plt.tight_layout()
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42033/3986200997.py:5: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Step 3: Add Realistic Noise

In [12]:
# Add mixed Gaussian + Poisson noise (realistic for microscopy)
I_fcpm_noisy = fcpm.add_fcpm_realistic_noise(
    I_fcpm_clean,
    noise_model='mixed',
    gaussian_sigma=0.05,  # 5% Gaussian noise
    seed=42
)

# Normalize
I_fcpm_noisy = fcpm.normalize_fcpm(I_fcpm_noisy, method='global')

# Visualize noisy FCPM
fig = fcpm.plot_fcpm_intensities(I_fcpm_noisy, z_idx=z_mid)
fig.suptitle('Noisy FCPM Intensities (5% noise)', y=1.02)
plt.tight_layout()
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42033/1551294888.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Step 4: Reconstruct Director Field

In [13]:
# Reconstruct using Q-tensor method with sign optimization
director_recon, info = fcpm.reconstruct(
    I_fcpm_noisy,
    fix_signs=True,
    verbose=True
)

print(f"\nReconstructed director shape: {director_recon.shape}")

Q-tensor reconstruction complete
Combined Sign Optimization

Phase 1: Chain propagation...
  After chain propagation: energy = 91705.79

Phase 2: Local flip refinement...
Initial gradient energy: 91705.79
  Iter 0: flipped 95 voxels, energy = 91705.79
  Iter 1: flipped 3 voxels, energy = 90242.89
  Iter 2: flipped 0 voxels, energy = 90238.95
Converged at iteration 2

Reconstructed director shape: (64, 64, 32)


### Step 5: Evaluate Reconstruction Quality

In [14]:
# Compute comprehensive metrics
metrics = fcpm.summary_metrics(director_recon, director_gt)

print("=" * 50)
print("Reconstruction Metrics")
print("=" * 50)
print(f"Angular error (mean):   {metrics['angular_error_mean_deg']:.2f} degrees")
print(f"Angular error (median): {metrics['angular_error_median_deg']:.2f} degrees")
print(f"Angular error (max):    {metrics['angular_error_max_deg']:.2f} degrees")
print(f"Angular error (90th):   {metrics['angular_error_90th_deg']:.2f} degrees")
print(f"Euclidean error (mean): {metrics['euclidean_error_mean']:.4f}")

Reconstruction Metrics
Angular error (mean):   10.85 degrees
Angular error (median): 11.02 degrees
Angular error (max):    89.17 degrees
Angular error (90th):   21.33 degrees
Euclidean error (mean): 0.1885


In [15]:
# Visual comparison: Ground Truth vs Reconstructed
fig = fcpm.compare_directors(director_gt, director_recon, z_idx=z_mid, step=2)
plt.tight_layout()
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42033/978022069.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [16]:
# Error map
fig, ax = plt.subplots(figsize=(8, 6))
fcpm.plot_error_map(director_recon, director_gt, z_idx=z_mid, ax=ax)
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42033/4178506343.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [17]:
# Error histogram
fig = fcpm.plot_error_histogram(director_recon, director_gt)
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42033/3785632564.py:3: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Noise Sensitivity Study

Let's see how reconstruction quality varies with noise level:

In [18]:
noise_levels = [0.01, 0.03, 0.05, 0.10, 0.15]
errors = []

for noise in noise_levels:
    # Add noise
    I_noisy = fcpm.add_fcpm_realistic_noise(
        I_fcpm_clean, noise_model='gaussian', gaussian_sigma=noise, seed=42
    )
    I_noisy = fcpm.normalize_fcpm(I_noisy)
    
    # Reconstruct
    d_recon, _ = fcpm.reconstruct(I_noisy, fix_signs=True, verbose=False)
    
    # Evaluate
    m = fcpm.summary_metrics(d_recon, director_gt)
    errors.append(m['angular_error_mean_deg'])
    print(f"Noise {noise*100:.0f}%: Mean angular error = {m['angular_error_mean_deg']:.2f} degrees")

Noise 1%: Mean angular error = 4.04 degrees
Noise 3%: Mean angular error = 7.38 degrees
Noise 5%: Mean angular error = 10.02 degrees
Noise 10%: Mean angular error = 17.08 degrees
Noise 15%: Mean angular error = 23.81 degrees


In [19]:
# Plot noise vs error
plt.figure(figsize=(8, 5))
plt.plot([n*100 for n in noise_levels], errors, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Noise Level (%)', fontsize=12)
plt.ylabel('Mean Angular Error (degrees)', fontsize=12)
plt.title('Reconstruction Error vs Noise Level', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_42033/1701288422.py:9: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Save Results

In [20]:
from pathlib import Path

output_dir = Path('../output/manual_workflow')
output_dir.mkdir(parents=True, exist_ok=True)

# Save director fields
fcpm.save_director_npz(director_gt, output_dir / 'director_ground_truth.npz')
fcpm.save_director_npz(director_recon, output_dir / 'director_reconstructed.npz')

# Save FCPM data
fcpm.save_fcpm_npz(I_fcpm_noisy, output_dir / 'fcpm_noisy.npz')

print(f"Results saved to: {output_dir.absolute()}")

Results saved to: /Users/narekmeloyan/PycharmProjects/FCPM-Simulation/examples/../output/manual_workflow
